Load the environment variables from a .env file

To get an api key, navigate to https://aistudio.google.com/apikey to create one. Then create a .env file in the same directory as this notebook with the following content:
```GOOGLE_API_KEY=your_api_key_here```


In [6]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

Single shot question answering with Gemini 2.0 Flash to check correct working of the code

In [39]:
from pydantic import BaseModel

class Recipe(BaseModel):
    medicijn: str
    categorie: str
    recept: str
    medische_vraag: str

def get_model(model_name: str = "gemini-2.0-flash") -> genai.GenerativeModel:
    return genai.GenerativeModel(model_name)

Create a function to get a recipe based on a medical question

In [35]:
import json
import re

def get_recipe(medical_question: str) -> Recipe:
    model = get_model()

    prompt = (
        "Beantwoord de volgende medische vraag met alleen een geldig JSON-object. "
        "Geef géén uitleg, alleen JSON, in exact dit formaat:\n\n"
        '{\n'
        '  "medicijn": "NAAM VAN HET MEDICIJN",\n'
        '  "categorie": "WAARVOOR IS HET",\n'
        '  "recept": "Rp. [Medicijnnaam] [Sterkte] [Vorm]\\n'
        'dtd. No. [Aantal]\\nS. [Instructie met duur van behandeling]",\n'
        '  "medische_vraag": "VRAAG DIE DE PATIËNT STELT"\n'
        '}\n\n'
        f"Medische vraag: {medical_question}"
    )

    response = model.generate_content(prompt)

    try:
        parsed = extract_json_from_text(response.text)
        return Recipe.model_validate(parsed)
    except Exception as e:
        print("Fout bij het parsen van het recept:", e)
        raise

def extract_json_from_text(text: str) -> dict:
    match = re.search(r'\{.*?\}', text, re.DOTALL)
    if match:
        return json.loads(match.group())
    else:
        raise ValueError("Geen JSON gevonden in het antwoord")


question = "Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?"
recipe = get_recipe(question)
print(recipe)


medicijn='Omeprazol' categorie='Protonpompremmer (maagzuurremmer)' recept='Rp. Omeprazol 20mg capsule\ndtd. No. 30\nS. 1 capsule per dag, 30 minuten voor het ontbijt' medische_vraag='Ik heb last van brandend maagzuur of zure reflux. Wat kan ik hiervoor nemen?'


In [36]:
import pandas as pd
filepath = "../data/unieke_vragen.json"
questions = pd.read_json(filepath)

questions

,0
0,Ik heb een keelontsteking. Heb ik antibiotica ...
1,Ik heb blaasontstekingklachten. Wat kan ik daa...
2,Ik heb een longontsteking. Welke antibiotica z...
3,Ik heb wondinfectieverschijnselen. Welk antibi...
4,Ik heb een tandvleesontsteking. Kan ik hier ie...
...,...
194,Ik heb slikproblemen. Zijn er medicijnen om de...
195,Ik heb spastische spieren door MS. Wat kan ik ...
196,Ik heb MS. Wat zijn mijn onderhoudsmedicijnen?
197,Ik heb een vergrote schildklier. Moet ik iets ...


In [37]:
import json

recipes = []

for question in questions.head(10)[0].tolist():
    try:
        recipe = get_recipe(question)
        recipes.append(recipe)
        print(f"Recipe generated: {recipe.model_dump()}")
    except Exception as e:
        print(f"Error generating recipe for question '{question}': {e}")

# Write the recipes to a JSON file
output_file = "../data/generated_recipes.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump([recipe.model_dump() for recipe in recipes], f, ensure_ascii=False, indent=4)


Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum', 'recept': 'Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. 3 maal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Ik heb een keelontsteking. Heb ik antibiotica nodig?'}
Recipe generated: {'medicijn': 'Nitrofurantoïne', 'categorie': 'Antibioticum bij blaasontsteking', 'recept': 'Rp. Nitrofurantoïne 100mg Capsules\ndtd. No. XXVIII\nS. 4dd 1 capsule gedurende 7 dagen', 'medische_vraag': 'Ik heb blaasontstekingklachten. Wat kan ik daartegen nemen?'}
Recipe generated: {'medicijn': 'Amoxicilline', 'categorie': 'Antibioticum', 'recept': 'Rp. Amoxicilline 500mg capsules\ndtd. No. 21\nS. driemaal daags 1 capsule gedurende 7 dagen.', 'medische_vraag': 'Ik heb een longontsteking. Welke antibiotica zijn geschikt?'}
Recipe generated: {'medicijn': 'Amoxicilline/Clavulaanzuur', 'categorie': 'Breed-spectrum antibioticum', 'recept': 'Rp. Amoxicilline/Clavulaanzuur 875/125mg tablet\ndtd. No. 14\nS. 2xdaags 1 tablet gedu

Check each recipe for correctness. Correctness is defined as follows:
- The recipe should contain a medicine name, strength, form, number of doses, and instructions.
- The medicine name should be a valid medicine name.
- The strength should be a valid strength.
- The form should be a valid form (e.g., tablet, capsule, liquid).
- The number of doses should be a valid number.
- The instructions should include the duration of the treatment.
- The indication should be relevant to the medical question.


In [44]:
def check_for_correctness(recipe: Recipe):
    model = get_model()
    prompt = (
        "Controleer of het onderstaande recept correct is volgens deze criteria:\n"
        "- Het recept moet een medicijnnaam, sterkte, vorm, aantal doses en instructies bevatten.\n"
        "- De medicijnnaam moet een geldige medicijnnaam zijn.\n"
        "- De sterkte moet een geldige sterkte zijn.\n"
        "- De vorm moet een geldige vorm zijn (bijv. tablet, capsule, vloeistof).\n"
        "- Het aantal doses moet een geldig getal zijn.\n"
        "- De instructies moeten de duur van de behandeling bevatten.\n"
        "- De indicatie moet relevant zijn voor de medische vraag.\n\n"
        "Geef het antwoord in geldig JSON-formaat met de volgende structuur:\n"
        '{\n'
        '  "medische_vraag": "<herhaal de medische vraag>",\n'
        '  "correct": true | false,\n'
        '  "fouten": [\n'
        '    "beschrijving van fout 1",\n'
        '    "beschrijving van fout 2"\n'
        '  ]\n'
        '}\n\n'
        f"Recept:\n{recipe.model_dump_json()}"
    )

    response = model.generate_content(prompt)

    return response.text


for recipe in recipes:
    result = check_for_correctness(recipe)
    print(result)

```json
{
  "medische_vraag": "Ik heb een keelontsteking. Heb ik antibiotica nodig?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb blaasontstekingklachten. Wat kan ik daartegen nemen?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb een longontsteking. Welke antibiotica zijn geschikt?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb wondinfectieverschijnselen. Welk antibioticum is nodig?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb een tandvleesontsteking. Kan ik hier iets voor slikken?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb een vaginale schimmelinfectie. Wat helpt hiertegen?",
  "correct": true,
  "fouten": []
}
```
```json
{
  "medische_vraag": "Ik heb een SOA. Welke behandeling is nodig?",
  "correct": false,
  "fouten": [
    "De medicijnnaam is niet gespecificeerd. \"Afhankelijk van SOA\" is geen geldige medicijn